In [1]:
import sys
sys.path.append("../..")
import os.path

from hydra import initialize, compose
from hydra.utils import instantiate

from pytorch_lightning.callbacks import ModelSummary
from pytorch_lightning import Trainer

import torch

In [2]:
def load_model(
        model_checkpoint: str,
) -> torch.nn.Module:
    model_dir = os.path.dirname(model_checkpoint)
    with initialize(config_path=os.path.join(model_dir, 'hydra')):
        cfg = compose('config.yaml')
    if "model" in cfg.keys():
        cfg["model"]["_target_"] = cfg["model"]["_target_"].replace(
            ".model.", ".network."
        )
        cfg["model"]["backbone"]["_target_"] = cfg["model"]["backbone"]["_target_"].replace(
            ".model.", ".network."
        )
        cfg["model"]["backbone"]["cartesian_weights_path"] = "../../" + cfg["model"]["backbone"]["cartesian_weights_path"]
        model = instantiate(
            cfg.model,
            optimizer_config=cfg.optimizer,
            _recursive_=False
        )
    else:
        cfg["network"]["backbone"]["cartesian_weights_path"] = "../../" + cfg["network"]["backbone"]["cartesian_weights_path"]
        model = instantiate(
            cfg.network,
            optimizer_config=cfg.optimizer,
            _recursive_=False
        )    
    state_dict = torch.load(model_checkpoint, map_location=torch.device("cpu"))
    model.load_state_dict(state_dict["state_dict"], strict=False)
    model = model.eval().cpu()
    model = model.requires_grad_(False)
    return model

In [3]:
network = load_model("../../data/models_jeanzay/unet_small/9/last.ckpt")

/tmp/ipykernel_989623/4017227567.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=os.path.join(model_dir, 'hydra')):


In [4]:
summary = ModelSummary(max_depth=5)
trainer = Trainer()
summary.on_fit_start(trainer, network)

/cerea_raid/users/finnt/usr/miniconda3/envs/subsinn/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cerea_raid/users/finnt/usr/miniconda3/envs/subsinn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [6]:
print(sum(p.numel() for p in network.backbone.parameters()))

3713536


In [7]:
print(sum(p.numel() for p in network.backbone.init_layer.parameters()))
print(sum(p.numel() for p in network.backbone.down_layers[0].pooling.parameters()))
print(sum(p.numel() for p in network.backbone.down_layers[0].out_layer[0].parameters()))
print(sum(p.numel() for p in network.backbone.down_layers[0].out_layer[1].parameters()))
print(sum(p.numel() for p in network.backbone.down_layers[0].out_layer[2].parameters()))
print("BN:", sum(p.numel() for p in network.backbone.bottleneck_layer.parameters()))
print(sum(p.numel() for p in network.backbone.up_layers[0].upscaling.parameters()))
print(sum(p.numel() for p in network.backbone.up_layers[0].out_layer[0].parameters()))
print(sum(p.numel() for p in network.backbone.up_layers[0].out_layer[1].parameters()))
print(sum(p.numel() for p in network.backbone.up_layers[0].out_layer[2].parameters()))
print(sum(p.numel() for p in network.backbone.out_layer.parameters()))

23296
295424
590336
590336
590336
BN: 590336
295168
295168
147712
147712
147712
